# Упражнение: Интерактивная диаграмма в Альтаире

Создайте диаграмму рассеяния с выбранными вами размерами набора данных фильмов, которые вы можете очистить, и сложенную гистограмму, которая фильтруется в соответствии с кистью.

Вот как должна выглядеть ваша составная гистограмма:


![Stacked Histogram](stacked_hist.png)

In [1]:
!pip install vega_datasets
!pip install altair.vegalite.v1

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement altair.vegalite.v1 (from versions: none)
ERROR: No matching distribution found for altair.vegalite.v1


In [2]:
from vega_datasets import data

# загрузили набор данных про машины
cars = data.cars()

cars.head()

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


In [3]:
import altair as alt 
import numpy as np


Задание:

Метки: mark_bar(), mark_text().

Столбцы данных: 'Cylinders', 'Displacement'.

Кодировки: opacity, text, tooltip.

In [5]:
single = alt.selection_point(on='mouseover', nearest=True)
car = alt.Chart(cars).mark_bar(opacity=0.3).encode(
    y=alt.Y('Cylinders:N').sort(field ='Cylinders', op = 'mean', order='descending'),
    x='mean(Displacement)'
    , color=alt.condition(single, 'Cylinders', alt.value('blue'))
    , tooltip='max(Displacement):N'
).add_params(
    single
)
text = car.mark_text( 
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='text:N'
).transform_calculate( # так как при использовании mean остаётся много знаков после запятой, форматируем 
    text = round(np.mean(cars['Displacement']),2).astype(str) 
)
car + text

alt.LayerChart(...)

5.2. Используя данные об автомобилях, создайте диаграмму рассеяния (*scatter-plot*), на которой *размер* (*size*) точек становится больше при наведении на них курсора.

In [6]:
# Переменные для интерактива при наведении
hover = alt.selection_point(on='mouseover', nearest=True, empty=False)
click = alt.selection_point(on='click', nearest=True, empty=False)

# Переменные для выбора второй переменной в scatter plot
dropdown = alt.binding_select(
    options=['Horsepower', 'Displacement', 'Weight_in_lbs', 'Acceleration'],
    name='X-axis column '
)
xcol_param = alt.param(
    value='Horsepower',
    bind=dropdown
)

scatter = alt.Chart(cars).mark_circle().encode(
    x=alt.X('x:Q').title(''),
    y='Miles_per_Gallon:Q',
    color='Origin:N'
).add_params(hover, click).properties().transform_calculate( # Реализация выбора второй переменной 
    x=f'datum[{xcol_param.name}]'
).add_params(
    xcol_param
)

# Реализация увеличения размера
highlight = scatter.mark_circle(opacity=0.90).encode(
    size=alt.condition(hover, alt.value(1000), alt.value(70))
)

(scatter + highlight)


alt.LayerChart(...)

5.3. Используя данные об автомобилях, создайте двухпанельную (*two-panel*) гистограмму (количество миль на галлон на одной панели, количество лошадиных сил на другой), где вы можете перетащить мышь, чтобы выбрать данные на левой панели, чтобы отфильтровать данные на второй панели.

In [21]:
interval = alt.selection_interval()
bar = alt.Chart(cars).mark_bar().encode( # создание гистограммы
    x='Miles_per_Gallon:Q'
    ,y='count()').add_params(
    interval
).transform_filter( # при выборе инервала так же фильтрует соседний график
    interval 
).interactive() # интерактивная диаграмма, которую можно приближать и отдалять

bar | bar.encode(x='Horsepower:Q')

alt.HConcatChart(...)

5.4. Измените приведенный выше пример диаграммы разброса и гистограммы, чтобы

- панорамировать и увеличивать диаграмму рассеяния;
- гистограмма отражала только те точки, которые видны в данный момент.

In [8]:
interval = alt.selection_interval(bind='scales')  # отражение только тех точек гистограммы, которые видно на диаграмме рассеивания
bar = alt.Chart(cars).mark_bar().encode(
    x='Miles_per_Gallon:Q'
    ,y='count()').add_params(
    interval
).transform_filter(
    interval
).interactive()

(scatter.interactive().add_params(interval) + highlight # панорамирование и выдиление точек
 & bar)

alt.VConcatChart(...)

5.5. Создайте диаграмму на основе самостоятельно сгенерированных данных и постройте кривые синуса и косинуса с помощью `transform_calculate`.

x = pd.DataFrame({'x': np.linspace(-(последняя цифра группы), (последняя цифра группы))})

In [9]:
import pandas as pd
x = pd.DataFrame({'x': np.linspace(-(18), (18))})
x.head(2)

,x
0,-18.000000
1,-17.265306


In [10]:
chart = alt.Chart(x).mark_line().encode(
    x='x',
    y='sin:Q'
).transform_calculate(
    cos='cos(datum.x)',
    sin='sin(datum.x)'
)
chart + chart.mark_line(color='firebrick').encode(y='cos:Q') # вфзов функций на одном графике

alt.LayerChart(...)

5.6. Используйте `transform_filter` на диаграмме задания 4 и удалите области графика, где значение кривой косинуса меньше значения кривой синуса.

In [11]:
interval = alt.selection_interval(bind='scales') 
chart = alt.Chart(x).mark_line().encode(
    x='x',
    y='sin:Q'
).add_params(
    interval
).transform_filter(
    interval
).transform_calculate(
    cos='cos(datum.x)', # расчёт косинуса
    sin='sin(datum.x)' # расчёт синуса
).interactive()
# вызов двух функций
(chart & chart.mark_line(color='firebrick').encode(y='cos:Q'))

alt.VConcatChart(...)

5.7. Используйте любимую визуализацию из предыдущего упражнения и настройте эстетику графика:
 

- настроить вид меток (`size`, `strokewidth` и т. д.).

- изменить оси и названия графика.

- изменить пределы `x` и `y`.

In [12]:
# Переменные для интерактива при наведении
hover = alt.selection_point(on='mouseover', nearest=True, empty=False)
click = alt.selection_point(on='click', nearest=True, empty=False)

# Переменные для выбора второй переменной в scatter plot
dropdown = alt.binding_select(
    options=['Horsepower', 'Displacement', 'Weight_in_lbs', 'Acceleration'],
    name='X-axis column '
)
xcol_param = alt.param(
    value='Horsepower',
    bind=dropdown
)
# Шестиугольнички
hexagon = "M0,-2.3094010768L2,-1.1547005384 2,1.1547005384 0,2.3094010768 -2,1.1547005384 -2,-1.1547005384Z"
scatter = alt.Chart(cars).mark_point(opacity=0.50,shape=hexagon).encode(
    x=alt.X('x:Q', title='Значение на выбор X', scale=alt.Scale(domain=(0, 200))).title(''),
    y=alt.Y('Miles_per_Gallon:Q', title='Miles_per_Gallon', scale=alt.Scale(domain=(0, 80))),
    color='Origin:N'
).add_params(hover, click).properties().transform_calculate( # Реализация выбора второй переменной 
    x=f'datum[{xcol_param.name}]'
).add_params(
    xcol_param
)

# Реализация увеличения размера
highlight = scatter.mark_point(opacity=0.80, shape=hexagon).encode(
    size=alt.condition(hover, alt.value(300), alt.value(30))
)

(scatter + highlight)

alt.LayerChart(...)

5.8. Используйте завершение табуляции в `alt.Chart.configure_`, чтобы увидеть различные параметры конфигурации, затем используйте `?`, чтобы увидеть документацию по функциям.

In [16]:
alt.Chart.configure_arc

<function altair.vegalite.v5.schema.mixins.ConfigMethodMixin.configure_arc(self, align: Union[ForwardRef('Align'), Literal['left', 'center', 'right'], ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, altair.utils.schemapi.UndefinedType] = Undefined, angle: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, float, altair.utils.schemapi.UndefinedType] = Undefined, aria: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, bool, altair.utils.schemapi.UndefinedType] = Undefined, ariaRole: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, str, altair.utils.schemapi.UndefinedType] = Undefined, ariaRoleDescription: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, str, altair.utils.schemapi.UndefinedType] = Undefined, aspect: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, bool, altair.utils.schemapi.UndefinedType] = Undefined, baseline: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, ForwardRef('TextBaseline'), ForwardRe

In [17]:
alt.Chart.configure_axis

<function altair.vegalite.v5.schema.mixins.ConfigMethodMixin.configure_axis(self, aria: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, bool, altair.utils.schemapi.UndefinedType] = Undefined, bandPosition: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, float, altair.utils.schemapi.UndefinedType] = Undefined, description: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, str, altair.utils.schemapi.UndefinedType] = Undefined, disable: Union[bool, altair.utils.schemapi.UndefinedType] = Undefined, domain: Union[bool, altair.utils.schemapi.UndefinedType] = Undefined, domainCap: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, ForwardRef('StrokeCap'), Literal['butt', 'round', 'square'], altair.utils.schemapi.UndefinedType] = Undefined, domainColor: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, NoneType, ForwardRef('Color'), ForwardRef('ColorName'), Literal['black', 'silver', 'gray', 'white', 'maroon', 'red', 'purple', 'fuchsia'

In [19]:
alt.Chart.configure_circle?

Signature:
alt.Chart.configure_circle(
    self,
    align: Union[ForwardRef('Align'), Literal['left', 'center', 'right'], ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, altair.utils.schemapi.UndefinedType] = Undefined,
    angle: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, float, altair.utils.schemapi.UndefinedType] = Undefined,
    aria: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, bool, altair.utils.schemapi.UndefinedType] = Undefined,
    ariaRole: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, str, altair.utils.schemapi.UndefinedType] = Undefined,
    ariaRoleDescription: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, str, altair.utils.schemapi.UndefinedType] = Undefined,
    aspect: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, bool, altair.utils.schemapi.UndefinedType] = Undefined,
    baseline: Union[ForwardRef('ExprRef'), ForwardRef('_Parameter'), dict, ForwardRef('TextBaseline'), ForwardRef('B